In [170]:
import pandas as pd
import yfinance as yf
import pandas as pd
import datetime
from ta.trend import EMAIndicator
import pandas_ta as pdta
import numpy as np
import ta

## Find Min Max of Swing

In [171]:
temp = None
MaxList = list()
MinList = list()
def FindMinMax(row):
    global temp,MinList,MaxList
    # print(row["Trend"],temp)
    if temp!= None :
        if row["Trend"] == 1 and temp == 1:
            MaxList.append(row["High"])
        elif row["Trend"] == -1 and temp == -1:
            MinList.append(row["Low"])
            
        elif row["Trend"] == -1 and temp == 1:
            MinList.clear()
            MinList.append(row["Low"])
            temp = row["Trend"]
            
        elif row["Trend"] == 1 and temp == -1:
            MaxList.clear()
            MaxList.append(row["High"])
            temp = row["Trend"]

        return min(MinList),max(MaxList)
            
    else:
        temp = row["Trend"]
        MaxList.append(row["High"])
        MinList.append(row["Low"])
    
    return min(MinList),max(MaxList)
    


# Data Gethering

In [172]:
df = pd.read_csv("Nifty50_5min.csv")
df_day = pd.read_csv("Nifty50_day.csv")
df.drop(columns=['Volume'], inplace=True)
df_day.drop(columns=['Volume'], inplace=True)

# Fetures Adding 

#### 1 Day Timeframe

In [173]:
super_trend = pdta.supertrend(high=df_day['High'], low=df_day['Low'], close=df_day['Close'], length=50, multiplier=4)
df_day['Trend'] = super_trend['SUPERTd_50_4.0']
df_day['Datetime'] = pd.to_datetime(df_day['Datetime'])
df_day['Date'] = df_day['Datetime'].dt.date

In [174]:
df_day["Swing"]=df_day.apply(FindMinMax,axis=1)
df_day = df_day.add_prefix('Day_')

In [175]:
df_day

,Day_Datetime,Day_Open,Day_High,Day_Low,Day_Close,Day_Trend,Day_Date,Day_Swing
0,2019-03-22 05:30:00,11549.20,11572.80,11434.55,11456.90,1,2019-03-22,"(11434.55, 11572.8)"
1,2019-03-25 05:30:00,11395.65,11395.65,11311.60,11354.25,1,2019-03-25,"(11434.55, 11572.8)"
2,2019-03-26 05:30:00,11375.20,11496.75,11352.45,11483.25,1,2019-03-26,"(11434.55, 11572.8)"
3,2019-03-27 05:30:00,11531.45,11546.20,11413.00,11445.05,1,2019-03-27,"(11434.55, 11572.8)"
4,2019-03-28 05:30:00,11463.65,11588.50,11452.45,11570.00,1,2019-03-28,"(11434.55, 11588.5)"
...,...,...,...,...,...,...,...,...
1235,2024-03-14 05:30:00,21982.55,22204.60,21917.50,22146.65,1,2024-03-14,"(21137.2, 22526.6)"
1236,2024-03-15 05:30:00,22064.85,22120.90,21931.70,22023.35,1,2024-03-15,"(21137.2, 22526.6)"
1237,2024-03-18 05:30:00,21990.10,22123.70,21916.55,22055.70,1,2024-03-18,"(21137.2, 22526.6)"
1238,2024-03-19 05:30:00,21946.45,21978.30,21793.10,21817.45,1,2024-03-19,"(21137.2, 22526.6)"


## 5 Min Timefra

In [176]:
df['Pivot'] = df[["High", "Low", "Close"]].mean(axis=1)
df['Candle'] = df.apply(lambda row: 'Green' if row['Close'] >= row['Open'] else 'Red', axis=1)
df['5EMA'] = EMAIndicator(close=df['Close'], window=5, fillna=False).ema_indicator()
df['15EMA'] = EMAIndicator(close=df['Close'], window=15, fillna=False).ema_indicator()
df['20EMA'] = EMAIndicator(close=df['Close'], window=20, fillna=False).ema_indicator()
df['50EMA'] = EMAIndicator(close=df['Close'], window=50, fillna=False).ema_indicator()
df['200EMA'] = EMAIndicator(close=df['Close'], window=200, fillna=False).ema_indicator()
df['RSI'] = ta.momentum.RSIIndicator(df['Close'],window=6).rsi()
super_trend = pdta.supertrend(high=df['High'], low=df['Low'], close=df['Close'], length=50, multiplier=4)
df['Trend'] = super_trend['SUPERTd_50_4.0']
df['CandleBody'] = abs(df['Close'] - df['Open'])
df.dropna(inplace=True)

In [177]:
df["Small Swing"]=df.apply(FindMinMax,axis=1)

In [178]:
df['Datetime'] = pd.to_datetime(df['Datetime'])
df['Day_Date'] = df['Datetime'].dt.date
# df

,Datetime,Open,High,Low,Close,Pivot,Candle,5EMA,15EMA,20EMA,50EMA,200EMA,RSI,Trend,CandleBody,Small Swing,Day_Date
199,2019-03-26 13:20:00,11389.20,11408.80,11387.90,11408.40,11401.700000,Green,11389.010925,11378.777846,11377.197546,11372.631130,11409.170179,93.170022,1,19.20,"(21137.2, 22526.6)",2019-03-26
200,2019-03-26 13:25:00,11408.50,11414.50,11406.70,11407.00,11409.400000,Red,11395.007283,11382.305615,11380.035875,11373.978929,11409.148585,89.229673,1,1.50,"(21137.2, 22526.6)",2019-03-26
201,2019-03-26 13:30:00,11406.80,11416.80,11406.80,11410.70,11411.433333,Green,11400.238189,11385.854913,11382.956268,11375.418971,11409.164022,90.503414,1,3.90,"(21137.2, 22526.6)",2019-03-26
202,2019-03-26 13:35:00,11410.20,11410.20,11403.80,11409.10,11407.700000,Red,11403.192126,11388.760549,11385.446147,11376.739795,11409.163385,85.270423,1,1.10,"(21137.2, 22526.6)",2019-03-26
203,2019-03-26 13:40:00,11408.90,11413.50,11403.20,11405.20,11407.300000,Red,11403.861417,11390.815480,11387.327467,11377.855882,11409.123948,72.935174,1,3.70,"(21137.2, 22526.6)",2019-03-26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92588,2024-03-20 15:05:00,21834.35,21840.25,21822.00,21836.90,21833.050000,Green,21837.404835,21846.422270,21849.504952,21849.578632,21895.019826,42.913658,-1,2.55,"(21813.95, 21930.9)",2024-03-20
92589,2024-03-20 15:10:00,21837.60,21843.05,21828.90,21842.25,21838.066667,Green,21839.019890,21845.900736,21848.814005,21849.291235,21894.494753,49.478799,-1,4.65,"(21813.95, 21930.9)",2024-03-20
92590,2024-03-20 15:15:00,21842.10,21856.35,21831.65,21855.10,21847.700000,Green,21844.379927,21847.050644,21849.412671,21849.519030,21894.102765,62.056033,-1,13.00,"(21813.95, 21930.9)",2024-03-20
92591,2024-03-20 15:20:00,21855.05,21858.45,21843.75,21847.00,21849.733333,Red,21845.253284,21847.044314,21849.182893,21849.420244,21893.634081,52.222068,-1,8.05,"(21813.95, 21930.9)",2024-03-20


In [179]:
merged_df = df_day.merge(df, on='Day_Date', how='inner')
merged_df.dropna(inplace=True)
# merged_df['DateTime'] = pd.to_datetime(merged_df['Day_Date'].astype(str) + ' ' + merged_df['Time'].astype(str))
merged_df.drop(['Day_Datetime'], axis=1, inplace=True)
merged_df.iloc[-2]

Day_Open                   21843.9
Day_High                   21930.9
Day_Low                    21710.2
Day_Close                  21839.1
Day_Trend                        1
Day_Date                2024-03-20
Day_Swing       (21137.2, 22526.6)
Datetime       2024-03-20 15:20:00
Open                      21855.05
High                      21858.45
Low                       21843.75
Close                      21847.0
Pivot                 21849.733333
Candle                         Red
5EMA                  21845.253284
15EMA                 21847.044314
20EMA                 21849.182893
50EMA                 21849.420244
200EMA                21893.634081
RSI                      52.222068
Trend                           -1
CandleBody                    8.05
Small Swing    (21813.95, 21930.9)
Name: 92392, dtype: object